In [1]:
from typing import List, Optional

import fire

from llama import Llama, Dialog

from tqdm.auto import tqdm
from time import sleep
from datasets import load_dataset, DatasetDict
from torch.utils.data import DataLoader, Dataset
import pickle as pkl
import os

# os.environ['CUDA_VISIBLE_DEVICES']="1,2"
import pandas as pd
import nltk
nltk.download('punkt')
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import torch
from datasets import load_dataset, DatasetDict, load_from_disk
from torch.utils.data import DataLoader, Dataset
import numpy as np
from tqdm import tqdm
import pandas as pd
from nltk import sent_tokenize
import math, re
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import sent_tokenize, word_tokenize
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from torchmetrics.text.rouge import ROUGEScore
from transformers import Trainer, TrainingArguments, pipeline
import argparse
import evaluate
# from styleformer import Styleformer
import warnings
warnings.filterwarnings("ignore")
import copy
import multiprocessing
import pickle as pkl
import openai
from dotenv import load_dotenv
import os
from time import sleep
from evaluate import load

/home/haskari/miniconda3/envs/acl/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to /home/haskari/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[2024-01-29 23:11:27,345] [INFO] [real_accelerator.py:161:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
def get_rouge(original_sentences_final,paraphrased_sentences_final):
    highlights = []
    model_s = []
    
    for j in original_sentences_final:
        highlights.append(' '.join(j))

    for k in paraphrased_sentences_final:
        model_s.append(' '.join(k))


    rouge = evaluate.load('rouge')

    print("==> Comparing generated summaries with gold summaries")
    results = rouge.compute(predictions=model_s, references=highlights)
    return results

In [3]:
def get_bertscore(original_sentences_final,paraphrased_sentences_final):
    
    highlights = []
    model_s = []


    for j in original_sentences_final:
        highlights.append(' '.join(j))

    for k in paraphrased_sentences_final:
        model_s.append(' '.join(k))
    
    bertscore = load("bertscore")
    
    results = bertscore.compute(predictions=model_s, references=highlights, lang="en", device='cuda:1')
    mean_precision=sum(results['precision'])/len(results['precision'])
    mean_recall=sum(results['recall'])/len(results['recall'])
    mean_f1=sum(results['f1'])/len(results['f1'])
    
    return mean_precision,mean_recall,mean_f1

In [4]:
with open('saved_data/cnn/deep1_cnn_train_42.pkl', 'rb') as f:
    deep1=pkl.load(f)

In [5]:
with open('saved_data/cnn/deep2_cnn_train_42.pkl', 'rb') as f:
    deep2=pkl.load(f)

In [6]:
with open('saved_data/cnn/llama.pkl', 'rb') as f:
    llama=pkl.load(f)

In [7]:
with open('saved_data/cnn/original.pkl', 'rb') as f:
    original=pkl.load(f)

In [8]:
original

[['The Palestinians signed the ICC\'s founding Rome Statute in January, when they also accepted its jurisdiction over alleged crimes committed "in the occupied Palestinian territory, including East Jerusalem, since June 13, 2014."',
  'Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis.',
  "Israel and the United States, neither of which is an ICC member, opposed the Palestinians' efforts to join the body.",
  'The inquiry will include alleged war crimes committed since June.'],
 ['A stray pooch in Washington State has used up at least three of her own after being hit by a car, apparently whacked on the head with a hammer in a misguided mercy killing and then buried in a field -- only to survive.',
  'Four days after her apparent death, the dog managed to stagger to a nearby farm, dirt-covered and emaciated, where she was found by a worker who took her to a vet 

In [52]:
original[0][1]

'Later that month, the ICC opened a preliminary examination into the situation in Palestinian territories, paving the way for possible war crimes investigations against Israelis.'

In [53]:
llama[0][1]

"The Palestinian authorities signed the ICC's founding Rome Statute in January, thereby acknowledging the court's authority to investigate and prosecute alleged crimes committed in the occupied Palestinian territories, including East Jerusalem, dating back to June 13, 2014."

In [51]:
mini=0.99
for i in range(len(original)):
    for j in range(len(original[i])):
        try:
            res=get_bertscore([original[i][j]],[llama[i][j]])
        except:
            pass
        if res[2]<mini:
            print(i,j)
            mini=res[2]
            print(mini)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0 0
0.924246609210968


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0 1
0.9238621592521667


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

1 1
0.9233897924423218


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


1 2
0.9012101292610168


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['ro

In [9]:
llama

[['(CNN)The Palestinian Authority officially became the 123rd member of the International Criminal Court on Wednesday, a step that gives the court jurisdiction over alleged crimes in Palestinian territories.',
  "The Palestinian authorities signed the ICC's founding Rome Statute in January, thereby acknowledging the court's authority to investigate and prosecute alleged crimes committed in the occupied Palestinian territories, including East Jerusalem, dating back to June 13, 2014.",
  "Neither Israel nor the United States, both of which are not members of the International Criminal Court (ICC), objected to the Palestinians' attempts to join the ICC.",
  'The International Criminal Court was set up in 2002 to prosecute genocide, crimes against humanity and war crimes.'],
 ['A wandering dog in Washington State has miraculously survived being struck by a car, despite suffering head injuries from a hammer attack and being buried in a field.',
  "That's according to Washington State Univer

In [11]:
res=get_rouge(original,llama)
res

==> Comparing generated summaries with gold summaries


{'rouge1': 0.6046319897000807,
 'rouge2': 0.34075127133367616,
 'rougeL': 0.4911236178123376,
 'rougeLsum': 0.48671103507437535}

In [12]:
res=get_bertscore(original,llama)
res

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.9207142353057861, 0.9227013528347016, 0.9216248691082001)

In [13]:
res=get_rouge(original,deep1)
res

==> Comparing generated summaries with gold summaries


{'rouge1': 0.8016979114269704,
 'rouge2': 0.6503130686789088,
 'rougeL': 0.7230080413080415,
 'rougeLsum': 0.721497476332927}

In [15]:
res=get_bertscore(original,deep1)
res

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.9500763654708863, 0.9358047723770142, 0.9428768575191497)

In [16]:
res=get_rouge(original,deep2)
res

==> Comparing generated summaries with gold summaries


{'rouge1': 0.7219889549496454,
 'rouge2': 0.47466018448382463,
 'rougeL': 0.5501737804229176,
 'rougeLsum': 0.5487110926931477}

In [17]:
res=get_bertscore(original,deep2)
res

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


(0.938933914899826, 0.9396045744419098, 0.9392397344112396)